## STEP 1

Prepare the dataset (events.csv)
- Ectract event-related text chuncks from the reports. A text chunck can be less than 100 words around a trigger phrase.
- E.g. When the trigger phrase is detected, 9 sentences are extracted, 4 sentences in front + the trigger sentence + 4 sentences after.
- Save the text chucks with 3 columns: Document_ID, Text_Chunck and Document_Label


In [37]:
import json 
import re
import os, glob #glob to open file folder
import csv #to read and write csv file 
import spacy
from spacy import displacy
import pandas as pd
from spacy.lang.en import English
from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher

In [81]:
df = pd.read_csv (r'C:\Users\charl\OneDrive - The University of Western Australia\Master DATA SCIENCE\2020\SEMESTER 2 2020\CITS5553 Data Science Capstone\report_groups.csv') 
charlie_files = df[df["group"] == 5]

In [2]:
# Import test report
# Opening JSON file
report = "C:/Users/charl/OneDrive - The University of Western Australia/Master DATA SCIENCE/2020/SEMESTER 2 2020/CITS5553 Data Science Capstone/data/wamex_xml/a071725_comannualrepc65-2005_draft2_13774268.json"
f = open(report,'r')  
report_sentences = json.load(f)
f.close()

In [60]:
folder_path = "C:/Users/charl/OneDrive - The University of Western Australia/Master DATA SCIENCE/2020/SEMESTER 2 2020/CITS5553 Data Science Capstone/"

def get_report(filename):
    with open(filename, 'r') as f:
        report_sentences = json.load(f)
        f.close()
        return report_sentences
    
# To begin with, we are just going to use the list of near-miss phrases provided by Paul as our trigger phrases
near_miss = "C:/Users/charl/OneDrive - The University of Western Australia/Master DATA SCIENCE/2020/SEMESTER 2 2020/CITS5553 Data Science Capstone/Near_miss_terms.txt"
near_miss_phrases = []

temp = open(near_miss,'r')
with temp as myfile:
    for line in myfile:
        if len(line) > 1:
            near_miss_phrases.append(line[:-2].split())
temp.close()

In [4]:
# Lets go through the sentences in the report and apply a cleaning function and store those sentences in a list
## Cleaning Function
def clean(text):
    text = text.strip('[(),- :\'\"\n]\s*').lower()
    #text = re.sub('([A-Za-z0-9\)]{2,}\.)([A-Z]+[a-z]*)', r"\g<1> \g<2>", text, flags=re.UNICODE)
    text = re.sub('\s+', ' ', text, flags=re.UNICODE).strip()
    text = re.sub('","', ' ', text, flags=re.UNICODE).strip()
    text = re.sub('-', ' ', text, flags=re.UNICODE).strip()
    #text = re.sub('\(', ' ', text, flags=re.UNICODE).strip()
    #text = re.sub('\)', ' ', text, flags=re.UNICODE).strip()
    text = re.sub('\/', ' ', text, flags=re.UNICODE).strip()
    text = text.replace("\\", ' ')
    text = ' '.join(text.split())
    
    if (text[len(text)-1] != '.'):
        text += '.'
    return text

In [7]:
#Create phrase matcher object
nlp = spacy.load('en_core_web_sm')

near_phrase_matcher = PhraseMatcher(nlp.vocab)

near_miss_phrases_string = []
for i in near_miss_phrases:
    near_miss_phrases_string.append(' '.join(i))

patterns = [nlp(text) for text in near_miss_phrases_string]

near_phrase_matcher.add("NearMissEvent",None, *patterns)


In [8]:
cleaned_report_sentences = []
for sentence in report_sentences:
    clean_sentence = clean(sentence)
    cleaned_report_sentences.append(clean_sentence)

In [9]:
doc = nlp(str(cleaned_report_sentences))
matched_phrases = near_phrase_matcher(doc)
for match_id, start, end in matched_phrases:
    string_id = nlp.vocab.strings[match_id]  
    span = doc[start:end]                   
    print(match_id, string_id, start, end, span.text)

15890475404117588905 NearMissEvent 4643 4644 encouraging


In [30]:
column_names = ["Document_ID", "Text_Chunk", "Trigger_words_activated", "Classification_vocab"]
event_extraction = pd.DataFrame(columns = column_names)

doc = nlp(str(cleaned_report_sentences))
matched_phrases = near_phrase_matcher(doc)

for match_id, start, end in matched_phrases:
     event_extraction_row = {"Document_ID":"Sample",
                             "Text_Chunk": doc[start-100:end+100],
                             "Trigger_words_activated": span.text,
                             "Classification_vocab": string_id}
     event_extraction = event_extraction.append(event_extraction_row, ignore_index=True)
    
event_extraction["Text_Chunk"][0]


margin of the lsb.', 'no fresh gravels presently come down the lower ord river system as far as the lsb.', 'there is no evidence of modern rechanging of gravels.', 'confidential page 20 5.', 'tidal currents prevent fine sediment from settling in the southern half of the lsb, but are not re working coarse sediments.', 'the fluvial gravels are a plausible host for a marine diamond deposit.', 'a lack of diamond recoveries, although not very encouraging, need not indicate a lack of a diamond deposit in the prospect.', 'a longer sampling programme will be required for fully testing the potential of the area.', 'the northern 25% of the lsb covers an area of approximately 9km2 with less than 4m overburden.', 'this area has not yet been sampled.', 'the northern lsb contains the most substantial areas of fluvial gravel and as such appears to be the best potential diamond target for continued exploration.', 'looking

In [79]:
#Create phrase matcher object
nlp = spacy.load('en_core_web_sm')

near_phrase_matcher = PhraseMatcher(nlp.vocab)

near_miss_phrases_string = []
for i in near_miss_phrases:
    near_miss_phrases_string.append(' '.join(i))

patterns = [nlp(text) for text in near_miss_phrases_string]

near_phrase_matcher.add("NearMissEvent",None, *patterns)
column_names = ["Document_ID","Text_Chunk_ID", "Text_Chunk", "Trigger_words_activated", "Classification_vocab"]
event_extraction = pd.DataFrame(columns = column_names)


for report_path, report_name in zip(charlie_files["path"],charlie_files['name']):
    file_path = folder_path + str(report_path)
    report_content = get_report(file_path)
    
    cleaned_report_sentences = []
    for sentence in report_content:
        clean_sentence = clean(sentence)
        cleaned_report_sentences.append(clean_sentence)
    
    doc = nlp(str(cleaned_report_sentences))
    matched_phrases = near_phrase_matcher(doc)
    for match_id, start, end in matched_phrases:
        count = 1
        event_extraction_row = {"Document_ID": str(report_name),
                                "Text_Chunk_ID": str(report_name + 'Chunk'+ str(count)),
                                "Text_Chunk": doc[start-100:end+100],
                                "Trigger_words_activated": doc[start:end],
                                "Classification_vocab": string_id}
        event_extraction = event_extraction.append(event_extraction_row, ignore_index=True)
        count +=1
    


In [80]:
event_extraction

,Document_ID,Text_Chunk_ID,Text_Chunk,Trigger_words_activated,Classification_vocab
0,a074375_annual report m59,a074375_annual report m59Chunk1,"(lake, moore, at, the, southern, end, of, a, m...",(broad),NearMissEvent
1,a074375_annual report m59,a074375_annual report m59Chunk1,"(than, the, back, dune, ., ', ,, ', photograph...",(broad),NearMissEvent
2,a074375_annual report m59,a074375_annual report m59Chunk1,"(inferred, resource, for, all, areas, of, 80mt...",(rising),NearMissEvent
3,a078309_weebacarry 2007-2008 annual report_146...,a078309_weebacarry 2007-2008 annual report_146...,"(', a, summary, of, the, historical, activity,...",(extensive),NearMissEvent
4,a078309_weebacarry 2007-2008 annual report_146...,a078309_weebacarry 2007-2008 annual report_146...,"(foliated, ., ', ,, ', interestingly, the, str...",(broad),NearMissEvent
...,...,...,...,...,...
267,a081220_yn_e53_1156_part_surrender_rpt_2008_12...,a081220_yn_e53_1156_part_surrender_rpt_2008_12...,"(mckay, and, miezitis, ,, 2001, ), ., ', ,, ',...",(extensive),NearMissEvent
268,a081220_yn_e53_1156_part_surrender_rpt_2008_12...,a081220_yn_e53_1156_part_surrender_rpt_2008_12...,"(below, the, water, table, ., ', ,, ', mineral...",(extensive),NearMissEvent
269,a081566_c21_2007_annual 2008_cr34109_ver_10415274,a081566_c21_2007_annual 2008_cr34109_ver_10415...,"(c21, 2007, woodline, project, annual, report,...",(broad),NearMissEvent
270,a081566_c21_2007_annual 2008_cr34109_ver_10415274,a081566_c21_2007_annual 2008_cr34109_ver_10415...,"(', ,, ', follow, up, rab, drilling, along, st...",(encouraging),NearMissEvent
